# Damping forces from models

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

In [ ]:
def get_dataset_name(vmm='vmm_linear', regression='force_regression', id=22773):
    jinja = "{{ vmm }}.{{ regression }}.{{ id }}.data_damping_forces"
    template = jinja.replace(r'{{','{').replace(r'}}','}')
    
    return eval(f"f'{template}'")

def get_dataset(vmm='vmm_linear', regression='force_regression', id=22773):
    
    name = get_dataset_name(vmm=vmm, regression=regression, id=id)
    return catalog.load(name)

def get_dataset_name_joined(vmm='vmm_linear', regression='force_regression', id=22773):
    jinja = "{{ vmm }}.{{ regression }}.joined.{{ id }}.data_damping_forces"
    template = jinja.replace(r'{{','{').replace(r'}}','}')
    
    return eval(f"f'{template}'")

def get_dataset_joined(vmm='vmm_linear', regression='force_regression', id=22773):
    
    name = get_dataset_name(vmm=vmm, regression=regression, id=id)
    return catalog.load(name)
    
    

In [ ]:
df_VCT = catalog.load('force_regression.data_scaled')

data = df_VCT[['fx','fy','mz','u','v','V','r','beta','delta','thrust','test type']].copy()
data['item'] = 'VCT'

vmms = ['vmm_martin']
model_test_ids = [22773]

for vmm in vmms:
    data_ = get_dataset(vmm=vmm)
    data_['item'] = get_dataset_name(vmm=vmm)
    data = data.append(data_)
    
for vmm in vmms:
    data_ = get_dataset_joined(vmm=vmm, regression='motion_regression')
    data_['item'] = get_dataset_name_joined(vmm=vmm, regression='motion_regression')
    data = data.append(data_)
    
for vmm in vmms:
    for model_test_id in model_test_ids:
        data_ = get_dataset(vmm=vmm, regression='motion_regression', id=model_test_id)
        data_['item'] = get_dataset_name(vmm=vmm, regression='motion_regression', id=model_test_id)
        data = data.append(data_)

In [ ]:
captive_plot(data, styles=['.','-'])

In [ ]:
data['item'].unique()

In [ ]:
data2 = data.groupby(by='item').filter(lambda x : x.iloc[0]['item'] in ['vmm_martin.force_regression.22773.data_damping_forces','vmm_martin.motion_regression.22773.data_damping_forces'])

In [ ]:
captive_plot(data2, styles=[':','-'])

In [ ]:
items=data.groupby(by='item')
df_force = items.get_group('vmm_martin.force_regression.22773.data_damping_forces')
df_motion= items.get_group('vmm_martin.motion_regression.22773.data_damping_forces')

In [ ]:
df_compare = df_force.copy()
forces = ['fx','fy','mz']
df_compare[forces] = df_force[forces]/df_motion[forces]
df_compare['item'] = 'compare'
df_compare['test type'] = df_force['test type']

In [ ]:
captive_plot(df_compare, styles=['-'])

In [ ]:
df_compare.sort_values(by='V').plot(x='V', y='fx')